In [1]:
import os
import random
import time
# Torch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler

# Torchvison
import torchvision.transforms as T
import torchvision.models as models
# from torchvision.datasets import CIFAR100, CIFAR10

# Utils
# import visdom
from tqdm import tqdm

# Custom
import models.resnet as resnet
import models.lossnet as lossnet
from config import *
from data.sampler import SubsetSequentialSampler
from data.fst_data import train_data,unlabeled_data, test_data

import timm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import datetime
# from torchsummary import summary as summary_

# Seed
random.seed("Inyoung Cho")
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True

##
# Data
# train_transform = T.Compose([
#     T.RandomHorizontalFlip(),
#     T.RandomCrop(size=32, padding=4),
#     T.ToTensor(),
#     T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]) # T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)) # CIFAR-100
# ])

# test_transform = T.Compose([
#     T.ToTensor(),
#     T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]) # T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)) # CIFAR-100
# ])

# cifar10_train = CIFAR10('../cifar10', train=True, download=True, transform=train_transform)
# cifar10_unlabeled   = CIFAR10('../cifar10', train=True, download=True, transform=test_transform)
# cifar10_test  = CIFAR10('../cifar10', train=False, download=True, transform=test_transform)



##
# Loss Prediction Loss
def LossPredLoss(input, target, margin=1.0, reduction='mean'):
    assert len(input) % 2 == 0, 'the batch size is not even.'
    assert input.shape == input.flip(0).shape
    
    input = (input - input.flip(0))[:len(input)//2] # [l_1 - l_2B, l_2 - l_2B-1, ... , l_B - l_B+1], where batch_size = 2B
    target = (target - target.flip(0))[:len(target)//2]
    target = target.detach()

    one = 2 * torch.sign(torch.clamp(target, min=0)) - 1 # 1 operation which is defined by the authors
    
    if reduction == 'mean':
        loss = torch.sum(torch.clamp(margin - one * input, min=0))
        loss = loss / input.size(0) # Note that the size of input is already halved
    elif reduction == 'none':
        loss = torch.clamp(margin - one * input, min=0)
    else:
        NotImplementedError()
    
    return loss


##
# Train Utils
iters = 0

#
def train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss, plot_data=None):
    models['backbone'].train()
    models['module'].train()
    global iters
    labeled_img = []
    
    for data in tqdm(dataloaders['train'], leave=False, total=len(dataloaders['train'])):
        inputs = data[0].to(device)
        labels = data[1].to(device)
        labeled_img.append(data[2])
        
        iters += 1

        optimizers['backbone'].zero_grad()
        optimizers['module'].zero_grad()

        
        scores, features = models['backbone'](inputs)
        target_loss = criterion(scores, labels)
        
        
        if epoch > epoch_loss:
            # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model.
            features[0] = features[0].detach()
            features[1] = features[1].detach()
            features[2] = features[2].detach()
            features[3] = features[3].detach()
        
            
        pred_loss = models['module'](features)
        pred_loss = pred_loss.view(pred_loss.size(0))


        m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        m_module_loss   = LossPredLoss(pred_loss, target_loss, margin=MARGIN)
        loss            = m_backbone_loss + WEIGHT * m_module_loss

        loss.backward()
        optimizers['backbone'].step()
        optimizers['module'].step()
        
    print(labeled_img)
    print(len(labeled_img))
    
def test(models, dataloaders, mode='val'):
    metrics = {}
    preds, labels = [], []
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()
    models['module'].eval()
    

    with torch.no_grad():
        for (inputs, label) in dataloaders[mode]:
            inputs = inputs.to(device)
            label = label.to(device)

            scores, _ = models['backbone'](inputs)
            # _, preds = torch.max(scores.data, 1)
            labels.extend(label.detach().tolist())
            preds.extend(scores.argmax(axis=1).detach().tolist())
        print(f" labels = {labels}")
        print(f" preds = {preds}")
    metrics['accuracy'] = accuracy_score(y_pred=preds, y_true=labels)
    metrics['f1_score'] = f1_score(y_pred=preds, y_true=labels, average='weighted')
    metrics['precision'] = precision_score(y_pred=preds, y_true=labels, average='weighted')
    metrics['recall'] = recall_score(y_pred=preds, y_true=labels, average='weighted')

            # print("labels " , labels)
            # print("preds " , preds)
    return metrics

#
def train(models, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss, plot_data):
    print('>> Train a Model.')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    for epoch in tqdm(range(num_epochs)):
            
        train_epoch(models, criterion, optimizers, dataloaders, epoch, epoch_loss, plot_data)
        schedulers['backbone'].step()
        schedulers['module'].step()

        # Save a checkpoint
        if False and epoch % 5 == 4:
            acc = test(models, dataloaders, 'test')
            if best_acc < acc:
                best_acc = acc
                torch.save({
                    'epoch': epoch + 1,
                    'state_dict_backbone': models['backbone'].state_dict(),
                    'state_dict_module': models['module'].state_dict()
                },
                '%s/active_resnet18_cifar10.pth' % (checkpoint_dir))
            print('Val Acc: {:.3f} \t Best Acc: {:.3f}'.format(acc, best_acc))
    print('>> Finished.')

#
def get_uncertainty(models, unlabeled_loader):
    models['backbone'].eval()
    models['module'].eval()
    uncertainty = torch.tensor([]).to(device)

    with torch.no_grad():
        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.to(device)
            # labels = labels.to(device)

            scores, features = models['backbone'](inputs)
            pred_loss = models['module'](features) # pred_loss = criterion(scores, labels) # ground truth loss
            pred_loss = pred_loss.view(pred_loss.size(0))

            uncertainty = torch.cat((uncertainty, pred_loss), 0)
    
    return uncertainty.cpu()


##
# Main
if __name__ == '__main__':
    # vis = visdom.Visdom(server='http://localhost', port=9000)
    plot_data = {'X': [], 'Y': [], 'legend': ['Backbone Loss', 'Module Loss', 'Total Loss']}
    start = time.time()
    
    #visualization에 사용
    acc_list = []
    f1_list = []
    
    for trial in range(TRIALS):
        # Initialize a labeled dataset by randomly sampling K=ADDENDUM=1,000 data points from the entire dataset.
        indices = list(range(NUM_TRAIN))
        random.shuffle(indices)
        labeled_set = indices[:ADDENDUM]
        unlabeled_set = indices[ADDENDUM:]
        
        print(f" labeled_set = {labeled_set}")
        # print(f" unlabeled_set = {unlabeled_set}")
        
        acc_list = []
        
        print(len(train_data))
        
        subset = SubsetRandomSampler(labeled_set)
        
        # print(f"subset = {subset}")
        
        # lst = [train_data[i] for i in subset]
        # print(f"lst = {lst}")
        
        train_loader = DataLoader(train_data, batch_size=BATCH, 
                                  sampler=subset, 
                                  num_workers = 4, pin_memory=True,  drop_last = True)
        test_loader  = DataLoader(test_data,num_workers = 4, batch_size=1)
        dataloaders  = {'train': train_loader, 'test': test_loader}
        

        print(len(dataloaders['train']))
        # Model
        resnet18    = resnet.ResNet50().to(device)
        # resnet18    = timm.create_model('resnet50', num_classes = 4).to(device)
        # print(f"resnet summary : {summary_(resnet18, (3, 32, 32))}")
        # resnet18 = timm.create_model("resnet50", num_classes = 4, pretrained=True).to(device)
        loss_module = lossnet.LossNet().to(device)
        models      = {'backbone': resnet18, 'module': loss_module}
        torch.backends.cudnn.benchmark = False

        # Active learning cycles
        for cycle in range(CYCLES):
            # Loss, criterion and scheduler (re)initialization
            criterion      = nn.CrossEntropyLoss(reduction='none')
            optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, 
                                    momentum=MOMENTUM, weight_decay=WDECAY)
            optim_module   = optim.SGD(models['module'].parameters(), lr=LR, 
                                    momentum=MOMENTUM, weight_decay=WDECAY)
            sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)
            sched_module   = lr_scheduler.MultiStepLR(optim_module, milestones=MILESTONES)

            optimizers = {'backbone': optim_backbone, 'module': optim_module}
            schedulers = {'backbone': sched_backbone, 'module': sched_module}

            # Training and test
            train(models, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL, plot_data)
            metrics = test(models, dataloaders, mode='test')
            print(f"accuracy : {metrics['accuracy']:.4f}")
            print(f"f1 score : {metrics['f1_score']:.4f}")
            print(f"precision : {metrics['precision']:.4f}")
            print(f"recall : {metrics['recall']:.4f}")
            print('Trial {}/{} || Cycle {}/{} || Label set size {}'.format(trial+1, TRIALS, cycle+1, CYCLES, len(labeled_set)))

            acc_list.append(metrics['accuracy'])
            f1_list.append(metrics['f1_score'])
            ##
            #  Update the labeled dataset via loss prediction-based uncertainty measurement

            # Randomly sample 10000 unlabeled data points
            random.shuffle(unlabeled_set)
            subset = unlabeled_set[:SUBSET]

            # Create unlabeled dataloader for the unlabeled subset
            unlabeled_loader = DataLoader(unlabeled_data, batch_size=BATCH, 
                                          sampler=SubsetSequentialSampler(subset), # more convenient if we maintain the order of subset
                                          num_workers = 4, pin_memory=True,  drop_last = True)

            # Measure uncertainty of each data points in the subset
            uncertainty = get_uncertainty(models, extractor, unlabeled_loader)

            # Index in ascending order
            arg = np.argsort(uncertainty)
            
            # Update the labeled dataset and the unlabeled dataset, respectively
            labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())
            unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

            # Create a new dataloader for the updated labeled dataset
            
            train_subset = SubsetRandomSampler(labeled_set)
            
            dataloaders['train'] = DataLoader(train_data, batch_size=BATCH, 
                                              sampler=train_subset, 
                                              num_workers = 4, pin_memory=True,  drop_last = True)
        
            print(f"train_subset : {train_subset}")
            
        # Save a checkpoint
        # torch.save({
        #             'trial': trial + 1,
        #             'state_dict_backbone': models['backbone'].state_dict(),
        #             'state_dict_module': models['module'].state_dict()
        #         },
        #         './cifar10/train/weights/active_resnet18_cifar10_trial{}.pth'.format(trial))
        end = time.time()
        
        timing = datetime.timedelta(seconds = end - start)
        
        print(f"working time = {timing}")
        print(f" acc_list = {acc_list}")
        print(f" f1_list = {f1_list}")

 labeled_set = [2265, 733, 543, 2419, 1253, 174, 2091, 1349, 1042, 910, 614, 1381, 273, 839, 888, 1998, 1564, 297, 454, 162, 902, 105, 2341, 357, 73, 1873, 1912, 1324, 2415, 954, 1924, 2581, 1146, 1480, 738, 1898, 1879, 2174, 1442, 1950, 293, 475, 1190, 53, 1090, 1047, 334, 1801, 594, 701]
2675
12
>> Train a Model.


100%|██████████| 1/1 [11:31<00:00, 691.01s/it]

[('/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220323-AT6T35EA-S015_inpaint_1 ADR_W1.0_H1.0_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/bz/train/220512-AT6T31EH-S019_inpaint_1 ADR_W1.8_H1.8_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/yd/train/220302-AT6T31E1-S015_inpaint_1 ADR_W2.9_H3.2_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220215-AT6T31E1-F128_inpaint_2 ADR_W1.3_H1.0_FS.jpg'), ('/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220314-AT6T31EH-D621_inpaint_2 ADR_W0.9_H0.8_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220324-AT6T31EH-D619_inpaint_2 ADR_W1.4_H1.0_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220221-AT6T31E1-F152_inpaint_5 ADR_W2.4_H2.6_FS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/bz/train/220414-AT6T31EH-M244_inpaint_1 ADR_W1.1_H0.9_BS.jpg'), ('/Users/ibyeong-gwon/Desktop/custom_data/hz/train/220318-AT6T31EH-M221_inpaint_1 ADR_W1.8_H1.5_BS.jpg', '/Users/ibyeong-gwon/Desktop/custom_data/bz/train/220228-A

ValueError: too many values to unpack (expected 2)